##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFLite 저작 도구

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/authoring"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

TensorFlow Lite Authoring API는 TensorFlow Lite와 `tf.function` 모델의 호환성을 유지하는 방법을 제공합니다.


## 설정

In [ ]:
import tensorflow as tf

## TensorFlow와 TensorFlow Lite 간의 호환성 문제

장치에서 TF 모델을 사용하려면 TFLite 인터프리터에서 사용할 수 있게 이를 TFLite 모델로 변환해야 합니다. 변환하는 동안 TFLite 기본 제공 op 세트에서 지원하지 않는 TensorFlow op로 인해 호환성 오류가 발생할 수 있습니다.

이것은 다소 성가신 문제입니다. 모델 저작 단계에서 이를 조기에 감지할 수 있는 방법은 무엇일까요?

다음 코드는 `converter.convert()` 호출에서 실패합니다.


In [ ]:
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")

In [ ]:
# Convert the tf.function
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [f.get_concrete_function()], f)
try:
  fb_model = converter.convert()
except Exception as e:
  print(f"Got an exception: {e}")

## 단순 타겟 인식 저작 방법

모델 저작 단계에서 TensorFlow Lite 호환성 문제를 감지하기 위해 Authoring API를 도입했습니다.

TFLite 호환성을 확인하기 위해 `tf.function` 모델을 래핑하려면 `@tf.lite.experimental.authoring.compatible` 데코레이터를 추가하기만 하면 됩니다.

그러면 모델을 평가할 때 호환성이 자동으로 확인됩니다.

In [ ]:
@tf.lite.experimental.authoring.compatible
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


TensorFlow Lite 호환성 문제가 발견되면 문제가 있는 op의 정확한 위치와 함께 `COMPATIBILITY WARNING` 또는 `COMPATIBILITY ERROR`가 표시됩니다. 이 예에서는 tf.function 모델에서 `tf.Cosh` op의 위치를 보여줍니다.

`<function_name>.get_compatibility_log()` 메서드로 호환성 로그를 확인할 수도 있습니다.

In [ ]:
compatibility_log = '\n'.join(f.get_compatibility_log())
print (f"compatibility_log = {compatibility_log}")

## 비호환성에 대한 예외 발생

`@tf.lite.experimental.authoring.compatible` 데코레이터에 옵션을 제공할 수 있습니다. `raise_exception` 옵션은 데코레이팅된 모델을 평가하려고 할 때 예외를 제공합니다.

In [ ]:
@tf.lite.experimental.authoring.compatible(raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
try:
  result = f(tf.constant([0.0]))
  print (f"result = {result}")
except Exception as e:
  print(f"Got an exception: {e}")

## "TF op 선택" 지정 방법

[Select TF ops](https://www.tensorflow.org/lite/guide/ops_select) 방법을 이미 알고 있다면 `converter_target_spec`을 설정하여 Authoring API에 이를 알릴 수 있습니다. [tf.lite.TFLiteConverter](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec) API에 사용할 동일한 [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter) 객체입니다.


In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec, raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


## GPU 호환성 확인

모델이 TensorFlow Lite의 [GPU 대리자](https://www.tensorflow.org/lite/performance/gpu)와 호환되도록 하려면 <a>tf.lite.TargetSpec</a>의 <code>experimental_supported_backends</code>를 설정할 수 있습니다.

다음 예제는 모델의 GPU 대리자 호환성을 보장하는 방법을 보여줍니다. 이 모델은 tf.slice 연산자 및 지원되지 않는 tf.cosh 연산자와 함께 2D 텐서를 사용하기 때문에 호환성 문제가 있습니다. 위치 정보와 함께 두 개의 `COMPATIBILITY WARNING`가 표시됩니다.

In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
target_spec.experimental_supported_backends = ["GPU"]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  y = tf.cosh(x)
  return y + tf.slice(x, [1, 1], [1, 1])

result = func(tf.ones(shape=(4,4), dtype=tf.float32))

## 더 읽어보기

자세한 내용은 다음을 참조하세요.

- [tf.function](https://www.tensorflow.org/api_docs/python/tf/function) API 문서
- [tf.function으로 성능 향상](https://www.tensorflow.org/guide/function)
- [TensorFlow Lite 변환기](https://www.tensorflow.org/lite/models/convert)
- [TensorFlow Lite 모델 분석기](https://www.tensorflow.org/lite/guide/model_analyzer)
- [TensorFlow Lite GPU 대리자](https://www.tensorflow.org/lite/performance/gpu)